# 👟 Behaviors

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import random

import ipywidgets as W
import networkx as nx
import pandas as pd

from ipyforcegraph.behaviors import NodeLabels, NodeSelection
from ipyforcegraph.forcegraph import ForceGraph

In [ ]:
selection = NodeSelection()

In [ ]:
labels = NodeLabels()

In [ ]:
fg = ForceGraph(behaviors=[labels, selection])
fg

In [ ]:
atlas = nx.graph_atlas_g()

In [ ]:
def update_graph(
    i: int = None,
    selected_color=None,
    not_selected_color=None,
) -> None:
    G = atlas[i if i is not None else random.randint(0, len(atlas))]
    nodes = list(G.nodes)
    fg.source.nodes = pd.DataFrame([{"label": f"Node {i}"} for i in nodes])
    fg.source.links = pd.DataFrame(
        {"source": nodes.index(source), "target": nodes.index(target)}
        for source, target in G.edges
    )
    selection.selected_color = selected_color or "#B3A369"
    selection.not_selected_color = not_selected_color or "#003057"


update_graph()

In [ ]:
@W.interact(
    selected_color=W.ColorPicker(value="#B3A369", description="selected"),
    not_selected_color=W.ColorPicker(value="#003057", description="not selected"),
)
def g(
    selected_color,
    not_selected_color,
    i=(0, len(atlas) - 1),
):
    update_graph(i, selected_color)

The selected nodes can be set from the kernel.

In [ ]:
selection.selected = [1, 2, 3]